# Vertical profile

This example shows how to plot a vertical profile with `roppy`.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import roppy

### Parameter settings

In [ ]:
roms_file = "data/ocean_avg_example.nc"
var = "temp"  # name of variable in NetCDF file
tstep = 3  # 4th time step
i, j = 100, 90  # Horizontal grid cell indices

## High level version

Let `roppy.SGrid` do the work. As `SGrid` is "lazy", we can not use a `with` statement.

In [ ]:
fid = Dataset(roms_file)
grid = roppy.SGrid(fid)

### Read the data

Read only the actual profile.

In [ ]:
profile_data = fid.variables[var][tstep, :, j, i]

 ### Plot the data

In [ ]:
plt.plot(profile_data, grid.z_r[:, j, i]);

## Low level version

Go directly at data from the given grid cell and use `roppy.sdepth` to extract the vertical structure.

### Read the data

Read the data and the vertical information needed to determine the depth of the `s` levels. Old ROMS files do not have the `Vtransform` information, in this case use `Vtransform = 1`.

In [ ]:
with Dataset(roms_file) as fid:
    # Read the data
    profile_data = fid.variables[var][tstep, :, j, i]
    long_name = fid.variables[var].long_name  # Save long_name for xlabel
    # Read necessary vertical information
    H = fid.variables['h'][j, i]
    C = fid.variables['Cs_r'][:]
    Hc = fid.variables['hc'].getValue()
    Vtransform = fid.variables.get('Vtransform', None)
# Handle default value for Vtransform
Vtransform = Vtransform.getValue() if Vtransform else 1

### Vertical discretization

Use `roppy` to compute the depth of the `s` levels.

In [ ]:
Z = -roppy.sdepth(H, Hc, C, Vtransform=Vtransform)

### Plot

Use matplotlib's `plot` function to show the profile

In [ ]:
plt.figure(figsize=(4,8))  # Make the plot higher
plt.plot(profile_data, Z, '-o')  # Mark the actual data points
plt.ylim([H, 0])           # Invert the vertical axis
plt.xlabel(long_name.capitalize())
plt.ylabel('Depth')
plt.grid(True)